In [30]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
# Load environment variables from .env file
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [25]:
def ready_prompt(user_query):

    prompt = f'''

    Prompt:
    You are a language model tasked with classifying user queries into one or more of the following categories:
    1.	Soil
    2.	Irrigation
    3.	Plant Diseases/Botany
    Each query can be associated with one or more of these labels. Your response should be formatted as a valid JSON object, specifying the number of labels assigned to the query and the labels themselves in the form of a Python list.
    Response Format:

    {{
        "RESPONSE": {{
            "answer": "<Number of labels assigned to the query>",
            "labels": <Labels according to you (should be a valid python list)>
        }}
    }}
    Example Query: "How can I improve soil fertility and what are the common diseases affecting tomatoes?"
    Example Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [1, 3]
        }}
    }}
    Your Task: For each user query, analyze the content and determine the appropriate labels (1 for Soil, 2 for Irrigation, and 3 for Plant Diseases/Botany). Provide the number of labels and the corresponding labels in the specified JSON format.
    Examples-

    Example 1: Single Label
    Query: "What type of soil is best for growing carrots?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "1",
            "labels": [1]
        }}
    }}
    Example 2: Single Label
    Query: "How often should I water my cucumber plants?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "1",
            "labels": [2]
        }}
    }}
    Example 3: Single Label
    Query: "What are the symptoms of powdery mildew on squash plants?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "1",
            "labels": [3]
        }}
    }}
    
    Example 4: Multi-Label
    Query: "What soil amendments are needed to improve water retention, and how do I prevent root rot in tomatoes?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [1, 3]
        }}
    }}
    Example 5: Multi-Label
    Query: "How can I set up a drip irrigation system and what are the benefits for soil health?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [1, 2]
        }}
    }}
    Example 6: Multi-Label
    Query: "What are the best practices for watering plants to avoid fungal diseases?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [2, 3]
        }}
    }}

    User Query: {user_query}


    '''
    
    return prompt

In [21]:
import google.generativeai as genai

genai.configure(api_key=gemini_api_key)

model = genai.GenerativeModel('gemini-1.5-pro')



In [37]:
import requests
def gpt4_with_for_multi_label_classification(user_query):
    
    prompt = ready_prompt(user_query=user_query)
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_api_key}"
    }
    
    payload = {
    "model": "gpt-4o",
    "messages": [
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": prompt
            },
        ]
        }
    ],
    # "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # print(response.json())
    res = response.json()['choices'][0]['message']['content']
    return res


In [27]:
def gemini_pro_with_for_multi_label_classification(user_query):
    
    prompt = ready_prompt(user_query=user_query)
    
    res = model.generate_content(
        contents=prompt
    )
    return res.text


In [31]:
res = gemini_pro_with_for_multi_label_classification('What are the benefits of using drip irrigation?')
res = res.replace('```', '').replace("\n", "").replace('json','')
json_res = json.loads(res)
json_res

{'RESPONSE': {'answer': '1', 'labels': [2]}}

In [36]:
res = gpt4_with_for_multi_label_classification('What are the benefits of using drip irrigation?')
res = res.replace('```', '').replace("\n", "").replace('json','')
json_res = json.loads(res)
json_res

{'id': 'chatcmpl-9f4kgVCX9BlASPUvUgzspkAo3sooh', 'object': 'chat.completion', 'created': 1719576742, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n    "RESPONSE": {\n        "answer": "1",\n        "labels": [2]\n    }\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 614, 'completion_tokens': 24, 'total_tokens': 638}, 'system_fingerprint': 'fp_ce0793330f'}


{'RESPONSE': {'answer': '1', 'labels': [2]}}

In [46]:

def check_simple_or_complexy_query(response):
    if len(response['RESPONSE']['labels']) > 1:
        return "COMPLEX"
    else:
        return "SIMPLE"

In [47]:
check_simple_or_complexy_query(json_res)

'SIMPLE'

In [51]:
def gemini_pro_with_for_generation_of_subproblems(complex_query):

    prompt = f'''

    You are tasked with breaking down a complex query into three specific categories: Soil, Irrigation, and Plant Diseases. A complex query consists of a single question that requires multiple operations to be performed simultaneously. For each category you determine to be relevant, rephrase the original user's question and break it into sub-problems. Ensure that the core meaning of the original user's question remains intact while dividing it into sub-problems.
    Your output should be in a well-defined valid JSON format as follows:
    {{
        "RESPONSE": {{
            "sub-problems": ["<list of sub-problems separated by commas as a Python list>"],
            "labels": ["<Python list of labels for each of the sub-problems devised>"]
        }}
    }}
    Examples:
    1.	Example Query: "How can I improve the soil quality, ensure proper irrigation, and prevent common diseases in my tomato plants?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What are the best practices to improve soil quality for tomato plants?",
                "How can I ensure proper irrigation for tomato plants?",
                "What are the common diseases that affect tomato plants and how can I prevent them?"
            ],
            "labels": ["Soil", "Irrigation", "Plant Diseases"]
        }}
    }}
    2.	Example Query: "What are the necessary soil amendments, irrigation techniques, and disease control measures for growing healthy cucumbers?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What soil amendments are necessary for growing healthy cucumbers?",
                "What irrigation techniques are best for cucumber plants?",
                "What disease control measures should I take for cucumber plants?"
            ],
            "labels": ["Soil", "Irrigation", "Plant Diseases"]
        }}
    }}
    3.	Example Query: "How do I prepare the soil and set up an efficient irrigation system for my new vineyard?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "How do I prepare the soil for a new vineyard?",
                "What steps should I take to set up an efficient irrigation system for a vineyard?"
            ],
            "labels": ["Soil", "Irrigation"]
        }}
    }}
    4.	Example Query: "What are the best soil treatments and how do I prevent diseases in my rose garden?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What are the best soil treatments for a rose garden?",
                "How do I prevent diseases in a rose garden?"
            ],
            "labels": ["Soil", "Plant Diseases"]
        }}
    }}
    5.	Example Query: "Can you suggest irrigation methods and disease prevention tips for my apple orchard?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What irrigation methods are suitable for an apple orchard?",
                "What are the best tips for preventing diseases in an apple orchard?"
            ],
            "labels": ["Irrigation", "Plant Diseases"]
        }}
    }}
    6.	Example Query: "How should I treat the soil and what irrigation system is ideal for my herb garden?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "How should I treat the soil for an herb garden?",
                "What irrigation system is ideal for an herb garden?"
            ],
            "labels": ["Soil", "Irrigation"]
        }}
    }}

    User query: {complex_query}
    '''
    
    res = model.generate_content(
        contents=prompt
    )
    return res.text

In [52]:
res = gemini_pro_with_for_generation_of_subproblems("What steps should I take to improve soil fertility, set up drip irrigation, and manage fungal diseases for my strawberry plants?")
res = res.replace('```', '').replace("\n", "").replace('json','')
json_res = json.loads(res)
json_res

{'RESPONSE': {'sub-problems': ['What steps should I take to improve soil fertility for my strawberry plants?',
   'How do I set up a drip irrigation system for my strawberry plants?',
   'How do I manage fungal diseases in strawberry plants?'],
  'labels': ['Soil', 'Irrigation', 'Plant Diseases']}}